In [2]:
import torch
import numpy as np

from torch import optim, nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as td
import torchvision.transforms as transforms
import torchvision.datasets as datasets


def custom_loader(batch_size, shuffle_test=False, data_path='./dataset/preprocessed'):
    # Add the necessary transforms
    # normalize = transforms.Normalize(mean=[0.024], std=[0.994])
    transform = transforms.Compose([
        transforms.Resize((48, 48)),  # Adjust this if your images are a different size
        # transforms.Grayscale(num_output_channels=1),  # Convert to grayscale
        transforms.ToTensor(),
        # normalize
    ])

    # Load your custom dataset
    train_dataset = datasets.ImageFolder(root=data_path + '/train', transform=transform)
    test_dataset = datasets.ImageFolder(root=data_path + '/validation', transform=transform)

    # Data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=shuffle_test, pin_memory=True)

    return train_loader, test_loader



class MultiLayerFCNet(nn.Module):
    def __init__(self,input_size, hidden_size, output_size):
        super().__init__()

        self.layer1=nn.Conv2d(3,32,4,padding=1,stride=1)
        self.B1 = nn.BatchNorm2d(32)
        self.layer2 = nn.Conv2d(32, 32, 4, padding=1, stride=1)
        self.B2 = nn.BatchNorm2d(32)
        self.Maxpool=nn.MaxPool2d(2)
        self.layer3 = nn.Conv2d(32, 64, 4, padding=1, stride=1)
        self.B3 = nn.BatchNorm2d(64)
        self.layer4 = nn.Conv2d(64, 64, 4, padding=1, stride=1)
        self.B4 = nn.BatchNorm2d(64)
        self.dropout = nn.Dropout(0.5)
        
        # New layers
        self.layer5 = nn.Conv2d(64, 128, 4, padding=1, stride=1)
        self.B5 = nn.BatchNorm2d(128)
        self.layer6 = nn.Conv2d(128, 128, 4, padding=1, stride=1)
        self.B6 = nn.BatchNorm2d(128)
        self.layer7 = nn.Conv2d(128, 256, 4, padding=1, stride=1)
        self.B7 = nn.BatchNorm2d(256)
        self.layer8 = nn.Conv2d(256, 256, 4, padding=1, stride=1)
        self.B8 = nn.BatchNorm2d(256)
        
        # Calculate the size for the fully connected layer after additional max-pooling layers
        # Assuming two max-pooling operations in the existing layers
        self.fc_size = 256   # Now this is 256 * 3 * 3
        self.fc = nn.Linear(self.fc_size, output_size)

    def forward(self, x):
        # Pass through existing layers
        x = F.leaky_relu(self.B1(self.layer1(x)))
        x = self.Maxpool(F.leaky_relu(self.B2(self.layer2(x))))
        x = F.leaky_relu(self.B3(self.layer3(x)))
        x = self.Maxpool(F.leaky_relu(self.B4(self.layer4(x))))
        
        # Pass through new layers
        x = F.leaky_relu(self.B5(self.layer5(x)))
        x = F.leaky_relu(self.B6(self.layer6(x)))
        x = self.Maxpool(F.leaky_relu(self.B7(self.layer7(x))))
        x = self.Maxpool(F.leaky_relu(self.B8(self.layer8(x))))
        
        x = x.view(x.size(0), -1)  # Flatten the tensor for the fully connected layer
        return self.fc(x)

if __name__ == '__main__':

    batch_size = 64
    test_batch_size = 64
    input_size = 3 * 48 * 48  # 1 channels, 48x48 image size
    hidden_size = 50  # Number of hidden units
    output_size = 4  # Number of output classes 4
    num_epochs = 10

    # train_loader, _ = cifar_loader(batch_size)
    # _, test_loader = cifar_loader(test_batch_size)
    train_loader, test_loader = custom_loader(batch_size, data_path='./dataset/preprocessed')
    # dataloader = DataLoader(dataset=IrisDataset('iris.data'),
    #                         batch_size=10,
    #                         shuffle=True)

    epochs = 50
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = MultiLayerFCNet(input_size, hidden_size, output_size)
    model = nn.DataParallel(model)
    model.to(device)
    #model.load_state_dict(torch.load('path'))

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    BestACC=0
    for epoch in range(epochs):
        running_loss = 0
        for instances, labels in train_loader:
            optimizer.zero_grad()

            output = model(instances)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(running_loss / len(train_loader))

        model.eval()
        with torch.no_grad():
            allsamps=0
            rightPred=0

            for instances, labels in test_loader:

                output = model(instances)
                predictedClass=torch.max(output,1)
                allsamps+=output.size(0)
                rightPred+=(torch.max(output,1)[1]==labels).sum()


            ACC=rightPred/allsamps
            print("epoch=",epoch)
            print('Accuracy is=',ACC*100)
            if ACC>BestACC:
                BestACC=ACC
                # torch.save(model.state_dict())
                # torch.save(model.state_dict(), 'path')
        model.train()


1.558601778380725
epoch= 0
Accuracy is= tensor(25.1497)
1.4397439932336613
epoch= 1
Accuracy is= tensor(25.5988)
1.4268574447047955
epoch= 2
Accuracy is= tensor(26.9461)
1.3948894763479427
epoch= 3
Accuracy is= tensor(29.1916)
1.3497106007167272
epoch= 4
Accuracy is= tensor(27.5449)
1.3255742666672687
epoch= 5
Accuracy is= tensor(30.9880)
1.2763737780707223
epoch= 6
Accuracy is= tensor(30.8383)
1.0301862115762672
epoch= 7
Accuracy is= tensor(47.6048)
0.8712253144809178
epoch= 8
Accuracy is= tensor(42.8144)
0.8453461491331762
epoch= 9
Accuracy is= tensor(46.1078)
0.8338194088060029
epoch= 10
Accuracy is= tensor(49.1018)
0.7742356086263851
epoch= 11
Accuracy is= tensor(54.4910)
0.764569421203769
epoch= 12
Accuracy is= tensor(56.8862)
0.7553873062133789
epoch= 13
Accuracy is= tensor(50.)
0.7434695509015298
epoch= 14
Accuracy is= tensor(52.0958)
0.7213280298271958
epoch= 15
Accuracy is= tensor(61.0778)
0.7317541716050129
epoch= 16
Accuracy is= tensor(45.6587)
0.7334960030049694
epoch= 17
A